In [1]:
import requests
import gzip
import pandas as pd
import networkx as nx
import numpy as np
import cPickle
import sys
from collections import Counter
import scipy.sparse
from scipy.sparse import coo_matrix, csc_matrix, csr_matrix, isspmatrix, issparse, isspmatrix_csr, isspmatrix_csc

%load_ext line_profiler
%load_ext autoreload
%autoreload 2

sys.path.insert(0, '/cellar/users/mikeyu/DeepTranslate/ddot')
import ddot
from ddot import Ontology

import pmc

import clixov_utils
import clixov_merge
import clique_maximal
import clique_maxcover
import mkl_spgemm
from clixov_merge import *
from clixov_utils import *

In [2]:
np.set_printoptions(linewidth=150)

In [3]:
# import warnings
# warnings.filterwarnings("error")

In [3]:
np.set_printoptions(precision=2)

In [5]:
# import os
# os.environ['NUMBA_DISABLE_JIT'] = '1'

# Load semantic similarity

## Yeast GO

In [15]:
f = np.load('yeast_go_20aug2017/resnik_ss.npz')
ss, ss_nodes = f['ss'], f['ss_nodes']

ont = Ontology.from_table('yeast_go_20aug2017/collapsed_go.20aug2017', clixo_format=True)

## Just the DNA repair branch of Yeast GO

In [6]:
f = np.load('/cellar/users/mikeyu/DeepCell/go_6oct2017_v2/resnik_ss_GO_0006281.npz')
ss, ss_nodes = f['ss'], f['ss_nodes']

ont = Ontology.from_table('/cellar/users/mikeyu/DeepCell/go_6oct2017_v2/collapsed_go_GO_0006281.propagated', clixo_format=True)

## Fan's HNeXO similarity

In [5]:
f = np.load('/cellar/users/mikeyu/DeepTranslate/hnexo/RFv2r3_square.npz')
ss, ss_nodes = f['rf'], f['genes']

# Iterate

In [16]:
ss = np.round(ss, 2)
np.triu_indices(ss.shape[0], k=1)
ss_flat_i, ss_flat_j = np.triu_indices(ss.shape[0], k=1)
ss_flat = ss[ss_flat_i, ss_flat_j]
print 'Distinct similarity scores:', np.unique(ss_flat).size

Distinct similarity scores: 408


In [17]:
## Initialize data structures
ss_argsort = np.argsort(-1 * ss_flat)

In [18]:
def count_unique_terms(ont, k):
    tmp = set([(tuple(sorted(ont.genes[g] for g in ont.term_2_gene[ont.terms[i]]))) for i in range(len(ont.terms)) if ont.term_sizes[i]==k])
    return len(tmp)

In [19]:
def collapse_identical(ont):
    ont = ont.propagate(gene_term=True, term_term=False, inplace=False)
    first = lambda x: x[0]
    import itertools
    term_groups = [[b for a, b in t_list] for _, t_list in itertools.groupby(sorted([(tuple(sorted(g)), t) for t, g in ont.term_2_gene.items()], key=first), key=first)]
    term_2_group = {t : '|'.join(t_list) for t_list in term_groups for t in t_list}
    return ont.rename(terms=term_2_group)

ont = collapse_identical(ont)

In [20]:
def expand_csr(X, new_shape):
    assert isspmatrix_csr(X)
    X.indptr = np.append(X.indptr, np.repeat(X.indptr[-1], new_shape[0] - X.shape[0]))
    X._shape = new_shape
    return X

In [35]:
def clixov(ss_argsort, n, beta, ont=None):
    dt = ss_flat[ss_argsort[0]]
    val_prev = dt
    dt_iter = 0
    n_genes = ss.shape[0]
    i_list, j_list = [], []

    n = ss.shape[0]
    G = csc_matrix((n,n), dtype=np.float32)
    X = np.zeros((n,n), dtype=np.float32)
    np.fill_diagonal(X, 1)
    X = csc_matrix(X)
    
    H = scipy.sparse.csr_matrix((n,n), dtype=np.bool)
    
    GX = csc_matrix((n,0), dtype=np.float32).toarray()
    prev_X_size = 0

    prev_merge = csr_matrix((n,n), dtype=np.int32)
#     prev_merge_i, prev_merge_j = np.array([], np.int32), np.array([], np.int32)
    
    if ont is not None:
        ont = ont.propagate(gene_term=True, term_term=False, inplace=False)
        term_sizes = np.array(ont.term_sizes)

    for ss_idx in ss_argsort:
        val = ss_flat[ss_idx]

        if val != val_prev or ss_idx==ss_argsort[-1]:
            prev_dt, dt, val_prev = dt, val_prev, val
            if ss_idx == ss_argsort[-1]:
                i, j = ss_flat_i[ss_idx], ss_flat_j[ss_idx]
                i_list.append(i)
                j_list.append(j)
            
            print '###########################################'
            start_clixov = time.time()
            expected_size = int(round(2**(-1 * dt) * n_genes))
            print 'dt_iter:', dt_iter, 'dt:', dt, 'prev_dt:', prev_dt, 'Expected size:', expected_size, round(2**(-1 * dt) * n_genes, 2)
            if ont is not None:
                print 'Ref terms:', (term_sizes==expected_size).sum()                
            print 'Total cliques:', X.shape[1], format_clique_sizes(X)

            dt_iter += 1
            print len(i_list), 'Added edges', G.sum() / 2, 'Current edges'

            # Only add edges that aren't in G yet
            tmp = as_dense_flat(G[i_list, j_list] == 0.)
            i_list, j_list = np.array(i_list)[tmp], np.array(j_list)[tmp]
            print "Edges not already in G:", i_list.size

            dG = coo_matrix((np.ones(i_list.size * 2, G.dtype),
                            (np.append(i_list, j_list), np.append(j_list, i_list))),
                            G.shape, copy=False)
            dG = csc_matrix(dG)
            newG = G + dG

            recent_X = X[:, prev_X_size:]
            recent_GX = as_dense_array(dot(newG, recent_X))
            GX = np.hstack([GX + as_dense_array(dot(dG, X[:,:prev_X_size])), recent_GX])
            
            # Reshape
            expand_csr(prev_merge, (X.shape[1], X.shape[1]))
            
#             if dt_iter==6:
#                 return X, GX, newG, G, dG, prev_merge_i, prev_merge_j, H
            import cPickle
            with open('/cellar/users/mikeyu/Data/clixov/dt_iters/%s.pkl' % dt_iter, 'wb') as f:
                cPickle.dump((X, GX, newG, beta, prev_merge, H, dG), f, protocol=cPickle.HIGHEST_PROTOCOL)
#                 np.savez('/cellar/users/mikeyu/Data/clixov/dt_iters/%s.npz' % dt_iter, X=X, GX=GX, newG=newG, beta=beta, prev_merge=prev_merge, H=H, dG=dG)
            
#             dX, merge_i, merge_j = calc_and_do_merge(X,
            dX, merge = calc_and_do_merge(X,
                                         X.toarray(),
                                         GX,
                                         newG,
                                         beta,
                                         prev_merge,
                                         H=H,
                                         dG=dG)
            assert_unique_cliques(dX)
            prev_merge += merge
            
#             prev_merge_i, prev_merge_j = np.append(prev_merge_i, merge_i), np.append(prev_merge_j, merge_j)
            print 'New cliques formed from merging:', dX.shape[1], format_clique_sizes(dX)
            
            unexplained = get_unexplained_edges(dX, dG)
            unexplained_n = unexplained.sum()
            print 'Unexplained edges:', unexplained_n
            assert unexplained_n == 0
            
            H = csr_matrix(update_subsumption(X, dX, H))
            assert np.all(H.data)
            
            prev_X_size = X.shape[1]
            G = newG

            if unexplained_n > 0:
                augment = clique_maxcover.max_clique_cover(unexplained, newG)
                augment = tuples_to_csc(augment, n, X.dtype)
                print 'Augmenting cliques:', augment.shape[1], format_clique_sizes(augment)
                assert augment.shape[1]>0
                X = csc_matrix(scipy.sparse.hstack([X, dX, augment]))
            else:
                X = csc_matrix(scipy.sparse.hstack([X, dX]))

            print 'prev_X_size:', prev_X_size        
            print 'prev_merge:', prev_merge.sum() / 2
            print 'Total iteration time:', time.time() - start_clixov
            i_list, j_list = [], []

        # Collect the gene pairs that have the same (or similar) similarities
    #     i, j = ss_idx / n_genes, ss_idx % n_genes
        i, j = ss_flat_i[ss_idx], ss_flat_j[ss_idx]
        i_list.append(i)
        j_list.append(j)
        
    return X

In [36]:
n = ss.shape[0]
beta = 1.0
# X, GX, newG, G, dG, prev_merge_i, prev_merge_j, H = clixov(ss_argsort, n, beta, ont)
# X, GX, newG, G, dG, prev_merge_i, prev_merge_j, unexplained, dX, merge_i, merge_j, augment = clixov(ss_argsort, n, beta)
# X = clixov(ss_argsort, n, beta)

# X = clixov(ss_argsort, n, 1.0)
%lprun -f clixov -f calc_and_do_merge -f get_unexplained_edges -f get_merge_mask1 -f get_thresholds_symm -f subsumption -f do_merge -f get_unexplained_edges -f fill_diagonal X = clixov(ss_argsort, n, 1.0, ont)
# %lprun -f get_merge_mask1 X = clixov(ss_argsort, n, beta, ont)

###########################################
dt_iter: 0 dt: 11.65 prev_dt: 11.65 Expected size: 2 2.01
Ref terms: 675
Total cliques: 6448 OrderedDict([(1, 6448)])
675 Added edges 0.0 Current edges
Edges not already in G: 675
onlytest: 1350
Test 675 out of 41570256 clique pairs:
Test merge time: 0.01939702034
To merge: 675
get_merge_mask1 time: 0.0234429836273
Iteration: 0 G/dG sum: 0 1350
	Transferred from dG to G: 0
	Clique search tree nodes / time: 1857 0.0119380950928
	Meta cliques: 675 OrderedDict([(2, 675)])
	Unexplained meta edges: 0
Merge cliques: 675 0.000772953033447 OrderedDict([(2, 675)])
orig_dG_genes: 1350 0.0197529792786
Merge cliques (cover): 675 OrderedDict([(2, 675)])
do_merge time: 0.0637290477753
New cliques formed from merging: 675 OrderedDict([(2, 675)])
Unexplained edges: 0.0
prev_X_size: 6448
prev_merge: 675
Total iteration time: 1.90729403496
###########################################
dt_iter: 1 dt: 11.07 prev_dt: 11.65 Expected size: 3 3.0
Ref terms: 466
Total 

get_merge_mask1 time: 0.109280109406
Iteration: 0 G/dG sum: 60896 10050
	Transferred from dG to G: 2204
	Clique search tree nodes / time: 4124 0.0379321575165
	Meta cliques: 224 OrderedDict([(2, 53), (3, 61), (4, 25), (5, 24), (6, 14), (7, 12), (8, 11), (9, 24)])
	Unexplained meta edges: 44
Iteration: 1 G/dG sum: 70902 44
	Transferred from dG to G: 0
	Clique search tree nodes / time: 46 0.0162341594696
	Meta cliques: 4 OrderedDict([(3, 2), (5, 1), (8, 1)])
	Unexplained meta edges: 0
Merge cliques: 228 0.000684022903442 OrderedDict([(3, 19), (4, 20), (5, 15), (6, 22), (7, 3), (8, 3), (9, 145), (10, 1)])
orig_dG_genes: 6496 0.0520091056824
Merge cliques (cover): 146 OrderedDict([(9, 145), (10, 1)])
do_merge time: 0.145467042923
New cliques formed from merging: 146 OrderedDict([(9, 145), (10, 1)])
Unexplained edges: 0.0
prev_X_size: 8827
prev_merge: 23845
Total iteration time: 2.9027569294
###########################################
dt_iter: 8 dt: 9.33 prev_dt: 9.48 Expected size: 10 10.0

	Clique search tree nodes / time: 3847 0.0547459125519
	Meta cliques: 271 OrderedDict([(2, 46), (3, 61), (4, 53), (5, 32), (6, 20), (7, 19), (8, 12), (9, 5), (10, 7), (11, 5), (12, 1), (13, 3), (14, 5), (15, 2)])
	Unexplained meta edges: 4
Iteration: 1 G/dG sum: 164424 4
	Transferred from dG to G: 0
	Clique search tree nodes / time: 7 0.0259420871735
	Meta cliques: 2 OrderedDict([(2, 1), (3, 1)])
	Unexplained meta edges: 0
Merge cliques: 273 0.000662803649902 OrderedDict([(3, 17), (4, 32), (5, 43), (6, 35), (7, 19), (8, 24), (9, 10), (10, 8), (11, 1), (12, 3), (13, 5), (14, 2), (15, 73), (16, 1)])
orig_dG_genes: 7878 0.0624730587006
Merge cliques (cover): 74 OrderedDict([(15, 73), (16, 1)])
do_merge time: 0.190356016159
New cliques formed from merging: 74 OrderedDict([(15, 73), (16, 1)])
Unexplained edges: 0.0
prev_X_size: 9503
prev_merge: 60787
Total iteration time: 3.2059378624
###########################################
dt_iter: 14 dt: 8.65 prev_dt: 8.75 Expected size: 16 16.05
Ref 

orig_dG_genes: 9846 0.0823791027069
Merge cliques (cover): 62 OrderedDict([(20, 58), (21, 3), (22, 1)])
do_merge time: 0.263088941574
New cliques formed from merging: 62 OrderedDict([(20, 58), (21, 3), (22, 1)])
Unexplained edges: 0.0
prev_X_size: 9838
prev_merge: 100151
Total iteration time: 3.37078595161
###########################################
dt_iter: 19 dt: 8.26 prev_dt: 8.33 Expected size: 21 21.03
Ref terms: 38
Total cliques: 9900 OrderedDict([(1, 6448), (2, 675), (3, 469), (4, 366), (5, 266), (6, 235), (7, 198), (8, 170), (9, 145), (10, 125), (11, 122), (12, 109), (13, 97), (14, 78), (15, 74), (16, 72), (17, 71), (18, 63), (19, 55), (20, 58), (21, 3), (22, 1)])
3486 Added edges 62493.0 Current edges
Edges not already in G: 3486
onlytest: 217756
Test 108878 out of 98000100 clique pairs:
Test merge time: 0.135685920715
To merge: 6255
get_merge_mask1 time: 0.145628213882
Iteration: 0 G/dG sum: 261110 12510
	Transferred from dG to G: 5098
	Clique search tree nodes / time: 3906 0

Total iteration time: 3.52319097519
###########################################
dt_iter: 24 dt: 7.95 prev_dt: 8.01 Expected size: 26 26.08
Ref terms: 27
Total cliques: 10082 OrderedDict([(1, 6448), (2, 675), (3, 469), (4, 366), (5, 266), (6, 235), (7, 198), (8, 170), (9, 145), (10, 125), (11, 122), (12, 109), (13, 97), (14, 78), (15, 74), (16, 72), (17, 71), (18, 63), (19, 55), (20, 58), (21, 41), (22, 35), (23, 33), (24, 42), (25, 34), (26, 1)])
3879 Added edges 85100.0 Current edges
Edges not already in G: 3879
onlytest: 295920
Test 147960 out of 101636642 clique pairs:
Test merge time: 0.163700819016
To merge: 7499
get_merge_mask1 time: 0.175580024719
Iteration: 0 G/dG sum: 350694 14998
	Transferred from dG to G: 6706
	Clique search tree nodes / time: 6821 0.107811927795
	Meta cliques: 743 OrderedDict([(2, 15), (3, 29), (4, 65), (5, 85), (6, 127), (7, 118), (8, 68), (9, 58), (10, 60), (11, 30), (12, 32), (13, 21), (14, 13), (15, 7), (16, 8), (17, 2), (18, 1), (20, 2), (21, 1), (22, 

...

	Unexplained meta edges: 52
Iteration: 1 G/dG sum: 2910898 52
	Transferred from dG to G: 6
	Clique search tree nodes / time: 17211 0.179776906967
	Meta cliques: 3374 OrderedDict([(5, 1), (6, 8), (7, 6), (8, 7), (9, 18), (10, 57), (11, 87), (12, 149), (13, 212), (14, 249), (15, 262), (16, 249), (17, 292), (18, 294), (19, 319), (20, 221), (21, 148), (22, 208), (23, 191), (24, 96), (25, 127), (26, 83), (27, 42), (28, 22), (29, 14), (30, 9), (31, 1), (32, 1), (35, 1)])
	Unexplained meta edges: 0
Merge cliques: 3060732 2.11256194115 OrderedDict([(3, 1), (4, 11), (5, 66), (6, 149), (7, 359), (8, 777), (9, 1420), (10, 1959), (11, 2736), (12, 3564), (13, 5302), (14, 7446), (15, 10226), (16, 14971), (17, 20325), (18, 28444), (19, 36889), (20, 44908), (21, 53537), (22, 61538), (23, 67948), (24, 77515), (25, 90301), (26, 102792), (27, 113421), (28, 124818), (29, 135391), (30, 144576), (31, 148839), (32, 152764), (33, 152759), (34, 149929), (35, 139710), (36, 132549), (37, 120669), (38, 109214), (

do_merge time: 13.7848930359
New cliques formed from merging: 2 OrderedDict([(120, 2)])
Unexplained edges: 0.0
prev_X_size: 11094
prev_merge: 1353217
Total iteration time: 17.5199048519
###########################################
dt_iter: 117 dt: 5.74 prev_dt: 5.75 Expected size: 121 120.65
Ref terms: 5
Total cliques: 11096 OrderedDict([(1, 6448), (2, 675), (3, 469), (4, 366), (5, 266), (6, 235), (7, 198), (8, 170), (9, 145), (10, 125), (11, 122), (12, 109), (13, 97), (14, 78), (15, 74), (16, 72), (17, 71), (18, 63), (19, 55), (20, 58), (21, 41), (22, 35), (23, 33), (24, 42), (25, 34), (26, 29), (27, 27), (28, 28), (29, 29), (30, 28), (31, 29), (32, 26), (33, 21), (34, 26), (35, 35), (36, 27), (37, 28), (38, 21), (39, 11), (40, 12), (41, 19), (42, 20), (43, 20), (44, 15), (45, 13), (46, 12), (47, 16), (48, 13), (49, 10), (50, 18), (51, 16), (52, 15), (53, 12), (54, 12), (55, 12), (56, 11), (57, 9), (58, 9), (59, 11), (60, 15), (61, 10), (62, 13), (63, 11), (64, 7), (65, 10), (66, 7), (

Merge cliques: 2436948 1.75050616264 OrderedDict([(4, 3), (6, 1), (7, 7), (8, 63), (9, 165), (10, 369), (11, 811), (12, 1575), (13, 2379), (14, 4031), (15, 6999), (16, 10574), (17, 15382), (18, 21711), (19, 29064), (20, 37751), (21, 46374), (22, 53394), (23, 59967), (24, 69756), (25, 78604), (26, 86640), (27, 90623), (28, 94151), (29, 98045), (30, 97624), (31, 95987), (32, 94020), (33, 90222), (34, 87652), (35, 83636), (36, 78913), (37, 75966), (38, 71572), (39, 69980), (40, 68778), (41, 66693), (42, 63008), (43, 57913), (44, 51830), (45, 46562), (46, 42339), (47, 38819), (48, 36238), (49, 33925), (50, 30664), (51, 28675), (52, 26396), (53, 24582), (54, 22852), (55, 21418), (56, 18903), (57, 16259), (58, 13625), (59, 11403), (60, 9811), (61, 8493), (62, 6957), (63, 5558), (64, 4987), (65, 4082), (66, 3570), (67, 3119), (68, 2570), (69, 2118), (70, 1861), (71, 1542), (72, 1366), (73, 1179), (74, 889), (75, 815), (76, 667), (77, 535), (78, 394), (79, 312), (80, 242), (81, 156), (82, 137)

	Unexplained meta edges: 96
Iteration: 1 G/dG sum: 3098272 96
	Transferred from dG to G: 4
	Clique search tree nodes / time: 8731 0.145704984665
	Meta cliques: 1863 OrderedDict([(5, 2), (6, 3), (7, 21), (8, 46), (9, 41), (10, 58), (11, 72), (12, 88), (13, 141), (14, 130), (15, 214), (16, 148), (17, 187), (18, 141), (19, 150), (20, 117), (21, 89), (22, 84), (23, 58), (24, 28), (25, 17), (26, 6), (27, 6), (28, 3), (29, 2), (30, 3), (31, 2), (32, 3), (33, 1), (35, 2)])
	Unexplained meta edges: 0
Merge cliques: 794980 0.482501029968 OrderedDict([(5, 9), (6, 35), (7, 94), (8, 244), (9, 407), (10, 671), (11, 1323), (12, 2642), (13, 3937), (14, 5474), (15, 7637), (16, 10316), (17, 13549), (18, 16623), (19, 20118), (20, 24402), (21, 27004), (22, 29136), (23, 32178), (24, 34234), (25, 38298), (26, 41055), (27, 41635), (28, 41918), (29, 43165), (30, 42808), (31, 41172), (32, 39847), (33, 36255), (34, 31804), (35, 27733), (36, 22657), (37, 18095), (38, 14127), (39, 11329), (40, 9651), (41, 8704),

	Unexplained meta edges: 80
Iteration: 1 G/dG sum: 3142414 80
	Transferred from dG to G: 0
	Clique search tree nodes / time: 610 0.112960100174
	Meta cliques: 91 OrderedDict([(5, 3), (6, 4), (7, 6), (8, 7), (9, 6), (10, 3), (11, 2), (12, 3), (13, 10), (14, 12), (15, 15), (16, 7), (17, 3), (18, 3), (19, 2), (20, 1), (21, 3), (25, 1)])
	Unexplained meta edges: 0
Merge cliques: 7659376 5.3347568512 OrderedDict([(5, 20), (6, 51), (7, 239), (8, 596), (9, 1387), (10, 2748), (11, 4933), (12, 7736), (13, 11653), (14, 16948), (15, 23099), (16, 30591), (17, 39777), (18, 50551), (19, 66321), (20, 84296), (21, 105991), (22, 129263), (23, 156411), (24, 183971), (25, 211887), (26, 238803), (27, 268021), (28, 296585), (29, 327861), (30, 352072), (31, 365931), (32, 378907), (33, 378953), (34, 370277), (35, 355223), (36, 339793), (37, 318881), (38, 296992), (39, 279603), (40, 260202), (41, 237614), (42, 211904), (43, 187574), (44, 168380), (45, 145820), (46, 126357), (47, 107457), (48, 89722), (49, 738

In [14]:
with open('tmp.npy') as f:
    X, dG = cPickle.load(f)

In [ ]:
import networkx as nx
import pygraphviz
from networkx.drawing.nx_agraph import graphviz_layout

In [96]:
# H = csr_matrix(subsumption(X))
# # H = csr_matrix(subsumption(X[:,[7, 6448]]))
# import igraph
# g = igraph.Graph(n=H.shape[0], edges=zip(*H.nonzero()), directed=True)
# assert g.is_dag()
# topo = g.topological_sorting(mode='out')
# print [(e.source, e.target) for e in g.es[:5]]
# # print list(topo)
# print topo.index(7), topo.index(6448)

# # import clique_maximal
# # clique_maximal.BK_hier_Gsep_wrapper(Gold, Gnew, H)

In [16]:
A, GA, G = X.toarray(), GX, newG
# A, GA, G = X.toarray()[:,[345, 703]], GX[:,[345, 703]], newG

threshold_loose, threshold_strict = get_thresholds(A, A, beta)
i_merge, j_merge = get_merge_mask1(A, GA, A, GA, G,
                                   threshold_loose, threshold_strict,
                                   symm=True,
                                   notest=(prev_merge_i, prev_merge_j))
# i_merge, j_merge = get_merge_mask1(A, GA, A, GA, G,
#                                    threshold_loose, threshold_strict,
#                                    symm=True)

Test 1044336 out of 25365003 clique pairs:
Merge time: 1.99702906609
To merge: 2755


In [ ]:
merge = do_merge(X, i_merge, j_merge, prev_merge_i, prev_merge_j)

# max_size = 0
# for i in range(merge.shape[1]):
#     clique = tuple(sorted(merge[:,i].nonzero()[0]))
#     max_size = max(max_size, len(clique))
#     if len(clique)>=4:
#         print '*', as_dense_array(G[clique,:][:,clique])
#     assert_clique(clique, G)
# print 'max_size:', max_size

In [54]:
newX, merge_i, merge_j = calc_and_do_merge(X,
                                           X.toarray(),
                                           GX,
                                           newG,
                                           beta,
                                           prev_merge_i,
                                           prev_merge_j,
                                           H=H,
                                           dG=dG)

onlytest: 416
Test 197 out of 106602 clique pairs:
Test merge time: 0.000344038009644
To merge: 113
get_merge_mask1 time: 0.00427007675171
Iteration: 0 G/dG sum: 448 226
[232 286]
[ 15 170 232]
	Transferred from dG to G: 60
	Search tree nodes / time: 74 0.0268878936768
	Meta cliques: 3 OrderedDict([(5, 2), (6, 1)])
	Unexplained meta edges: 0
Merge cliques: 3 0.000420093536377 OrderedDict([(8, 3)])
orig_dG_genes: 116 0.00335502624512
Merge cliques (cover): 3 OrderedDict([(8, 3)])
do_merge time: 0.0528020858765


# asdf

In [ ]:
with open('tmp.npy', 'wb') as f: cPickle.dump((Gold, Gnew, H), f)

In [102]:
np.all(subsumption(X).toarray() == H.toarray())

False

In [111]:
print H[310,:].nonzero()[1]
print H[315,:].nonzero()[1]

[]
[]


In [14]:
with open('tmp.npy') as f:
    Gold, Gnew, H = cPickle.load(f)

In [75]:
C, CP, CN, tree = clique_maximal.BK_dG_py(Gold, Gnew)
cliques_csc = cliques_to_csc(C, CP, CN, n)
print cliques_csc.shape
for i in range(cliques_csc.shape[1]):
    print cliques_csc[:,i].nonzero()[0]

(304, 3)
[  0 104 136 153 221 232 257 286]
[ 17 128 138 148 231 233 249 270]
[ 17  54 147 162 189 196 245 274]


In [76]:
[g for t, g in ont.term_2_gene.items() if len(g)==8]

[[0, 104, 136, 153, 221, 232, 257, 286],
 [17, 128, 138, 148, 231, 233, 249, 270],
 [17, 54, 147, 162, 189, 196, 245, 274]]

In [ ]:
C, CP, CN, tree = clique_maximal.BK_hier_dG_py(Gold, Gnew, H)
cliques_csc = cliques_to_csc(C, CP, CN, n)
print cliques_csc.shape

In [74]:
Gold, Gnew = clixov_utils.get_test_new_network('cluster', k=200, r=10, s=10, verbose=False)
%time cliques, cliques_indptr, cliques_n, tree_size = clique_maximal.BKPivotSparse2_Gnew_wrapper(Gold, Gnew)
cliques = cliques_to_csc(cliques, cliques_indptr, cliques_n, Gold.shape[0])

tmp1 = csc_to_cliques_list(cliques)
%time tmp2 = clique_maximal.get_cliques_igraph(Gold.shape[0], Gold + Gnew, Gnew, input_fmt='matrix')
assert set(tmp1) == set(tmp2)
len(tmp1)

CPU times: user 0 ns, sys: 4 ms, total: 4 ms
Wall time: 2.45 ms
CPU times: user 152 ms, sys: 0 ns, total: 152 ms
Wall time: 149 ms


37

In [145]:
X_set = set([])
for i in as_dense_flat(X.sum(0)==4).nonzero()[0]:
    X_set.add(tuple(sorted(X[:, i].nonzero()[0])))
len(X_set)

386

In [144]:
ont_set = set([tuple(sorted(ont.term_2_gene[t])) for t, s in zip(ont.terms, ont.term_sizes) if s==4])
len(ont_set)

303